### Dependencies

In [1]:
import os
from pathlib import Path
import torch
import imageio
from IPython.display import HTML, Image
import warnings
warnings.filterwarnings("ignore")

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

from pygpudrive.env.config import EnvConfig, RenderConfig, SceneConfig, SelectionDiscipline
from pygpudrive.env.env_torch import GPUDriveTorchEnv

## Configurations

### Environment 
- only control valid agents up to a max of 32
- initialization steps: 10
- dynamics_model (TODO): `StateDynamics`
    - 5D action `(x, y, yaw, velocity x, velocity y)`

In [28]:
scene_config = SceneConfig(
    path="data/examples", 
    num_scenes=1,
    discipline=SelectionDiscipline.K_UNIQUE_N,
    k_unique_scenes=1,
)

env_config = EnvConfig(
    init_steps=11, 
)

render_config = RenderConfig()

### Make environment


In [29]:
env = GPUDriveTorchEnv(
    config=env_config,
    scene_config=scene_config,
    render_config=render_config,
    max_cont_agents=32, # Maximum number of agents to control per scene
    device="cpu",
)


--- Ratio unique scenes / number of worls =         1 / 1 ---



In [30]:
# Sanity check: We have a warmup period of 10 steps, so the first step should be 11
env.reset();
env.step_in_episode

11

### Load trained vbd model

In [31]:
# Load model
from vbd.sim_agent.sim_actor import VBDTest, sample_to_action

ckpt_path = 'vbd/weights/epoch=18.ckpt'

model = VBDTest.load_from_checkpoint(ckpt_path, map_location=torch.device('cpu'))
_ = model.cuda()
_ = model.eval()

### Align data format

```Python 
sample = dataset.process_scenario(
  current_state, 
  current_index = current_state.timestep,
  use_log=False,
  selected_agents=selected_agents, # override the agent selection by distance to the ego
)
batch = dataset.__collate_fn__([sample])
pred = model.sample_denoiser(batch)
traj_pred = pred['denoised_trajs'].cpu().numpy()[0]
```

In [ ]:
# torch.Size([1, 32, 11, 8])

sample = {}

sample['agent_history'] =

## Rollout with goal guidance

In [23]:
replan_freq=80 # Roll out every X steps 80 means openloop
model.early_stop=0 # Stop Diffusion Early From 100 to X
model.skip = 1 # Skip Alpha 
model.reward_func = None

# Reset the environment
init_state = env.reset()

current_state = init_state



In [24]:
init_state

tensor([[[ 8.6767e-02,  1.5411e-01,  2.0509e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 6.2707e-02,  1.4943e-01,  2.0013e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.8664e-01,  2.4412e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [-3.1737e-08, -2.6657e-06, -6.6207e-05,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 6.3080e-32,  0.0000e+00,  4.2039e-45,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]])